In [ ]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from util import printFiles, depthFirstPrint

In [ ]:
credentials_file = "client_secret.json"
scopes = [ "https://www.googleapis.com/auth/drive.readonly" ]

In [ ]:
flow = InstalledAppFlow.from_client_secrets_file(
  credentials_file, scopes=scopes
)
credentials = flow.run_local_server(port=0)

In [ ]:
drive_service = build("drive", "v3", credentials=credentials)
files = drive_service.files()
fields = "id, name, parents, ownedByMe, sharingUser" # Used by files.get
listFields = f'nextPageToken, files({fields})'       # Used by files.list

In [ ]:
# Get list of files returned by drive
file_list = list()
next_list = files.list(fields=listFields).execute()
file_list.extend(next_list['files'])
while 'nextPageToken' in next_list and next_list['nextPageToken']:
  next_list = files.list(fields=listFields, pageToken=next_list['nextPageToken']).execute()
  file_list.extend(next_list['files'])
fileById = {f['id']: f for f in file_list}

In [ ]:
# Get metadadata on parents not included in files.list (just 'My Drive')
fids = list(fileById.keys())
while len(fids) > 0:
    fid = fids.pop()
    f = fileById[fid]
    if 'parents' in f:
        [pid] = f['parents']
        if pid not in fileById:
            fids.append(pid)
            fileById[pid] = files.get(fileId=pid, fields=fields).execute()

In [ ]:
# Initialize and assign list of children
for f in fileById.values():
  f['children'] = list()
for fid, f in fileById.items():
  if 'parents' in f:
    [pid] = f['parents']
    fileById[pid]['children'].append(fid)

In [ ]:
# Find files without parents ()
myRoots = list()      # Just 'My Drive'
otherRoots = list()   # Files shared with me.
for f in fileById.values():
    if 'parents' not in f:
        if f['ownedByMe']:
            myRoots.append(f)
        else:
            otherRoots.append(f)

In [ ]:
for root in myRoots + otherRoots:
    print(f'Printing {root["id"]}')
    depthFirstPrint(root['id'], fileById)

In [ ]:
myRoots

In [ ]:
otherRoots

In [ ]:
print(f'There are {len(noParent)} files with no parent')
printFiles(noParent)

In [ ]:
print(f'There are {len(sharedWithMe)} files shared with me')
printFiles(sharedWithMe)

In [ ]:
print(f'There are {len(externalParent)} files with an external parent')
printFiles(externalParent)

In [ ]:
print(f'There are {len(sharedWithOther)} files that I have shared with others')
printFiles(sharedWithOther)

In [ ]:
files.listLabels(fileId=p_id).execute()

In [ ]:
f = files.get(fileId=file0['id']).execute()
f